# SymScore (Symbolic Regression-Based Clinical Score Generator)

In [21]:
import pandas as pd
import numpy as np
import re, copy, time
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt


from sklearn.cluster import KMeans

import sklearn.metrics as metrics

from tqdm.notebook import tqdm
from gplearn.functions import make_function
from gplearn.fitness import make_fitness
from gplearn.genetic import SymbolicRegressor as SR
from gplearn.genetic import SymbolicClassifier as SC
from sympy import sympify,sin, cos
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from skopt import gp_minimize
from skopt.space import Real, Integer
import itertools
import re
import pickle

# Input the excel file containing survey response

Here, simply input the name of the Excel file containing the survey response. For example:

filename = "survery_response.xlsx"

The Excel file must contain the input variables as column names (e.g age, weight, BMI, etc) and the target variables. For regression tasks, the target variable is the total score of the input variables and for the classification tasks, the target variable is the diagnosis, where the entry is 0 for not having the disease and 1 as not having the disease. 

Furthermore, the specific task should be specified here. For the variable task:

Write 0 for regression task and write 1 for classification task. 

In [22]:
filename = "MCQI score only_kor_원본.xlsx"
task = 0

# Inputs for regression task

(Ignore this if your task is a classification task)
Here, please input the necessary information regarding the shortened questionnaire responses.

1. Selected categorical variables
    
    For categorical questions where the responses increase with increasing severity, please provide the column names for the selected categorical variables and the response list.
    
    For example, if the selected questions have column names 23, 28, 39, 51, 58, 60, then write selected_categorical_variables = [23, 28, 39, 51, 58, 60]
    
2. Categorical response list
    
    If the possible responses are 1 - No, 2 - Slight, 3 - Moderate, 4 - Severe, then write down
categorical_response_list = [1, 2, 3, 4]

3. Maximum number of items in questionnaire

    Please also input the maximum number of items in the questionnaire. For example, for the MCQI, there are 60 total items so max_items = 60.

    

In [23]:
# Inputs for regression task
if task == 0:
    selected_categorical_variables = [23, 28, 39, 51, 58, 60]
    categorical_response_list = [1, 2, 3, 4]
    max_items = 60

# Inputs for classification task

(Ignore this if your task is a regression task) 
Here, please input the necessary information regarding the shortened questionnaire responses.

1. Diagnosis column 

    Here, indicate the column name that contains the target value. In this case, the diagnosis for COMISA has column name 1. OSA so we write diagnosis_column = "1. OSA".
    
2. Selected categorical variables 

    For categorical questions where the responses increase with increasing severity, please provide the column names for the selected categorical variables and the response list.
    For example, if the selected questions have column names "ISI1a", "ISI1b", "ISI1c", "ISI2", "ISI5", then write categorical_items = ["ISI1a", "ISI1b", "ISI1c", "ISI2", "ISI5"].
    
    
    
3. Categorical response list
    
    Additionally, if the possible responses are 1 - No, 2 - Slight, 3 - Moderate, 4 - Severe, then write down categorical_response_list = [1, 2, 3, 4]
    
4. Binary variables

    If the questions contain binary variables, then indicate it. 
    For example, for the variable sex, write
    categorical_items_binary = ['sex'] and 
    binary_variables = ['Male', 'Female']
    
5. Float variablles

    Similarly indicate the float variables by writing the column names. 
    For example here, float_items = ["age", "weight", "BMI"].
    
6. Step size 

    Please also indicate the step sizes for the float items. These step sizes indicate how accurate you want your response grouping to be. For example, a step size of 1 will check if the grouping is optimal for every 1-step increase in value, while a step size of 10 will check for groupings after every 10 unit increase.

    

In [24]:
# Inputs for classification task
if task == 1:
    diagnosis_column = "9. COMISA"
    categorical_items = ["ISI1a", "ISI1b", "ISI1c", "ISI2", "ISI5"]
    categorical_response_list = [0, 1, 2, 3, 4]
    categorical_items_binary = ['sex']
    binary_variables = ["Male", "Female"]
    float_items = ["age", "weight", "BMI"]
    user_defined_step_sizes = {
            'age': 10,
            'weight': 10,
            'BMI': 5
        }


# Start of SymScore framework

In [25]:
df = pd.read_excel(filename) 
df= df.replace(999,-1) #removing missing values

In [26]:
# Necessary inputs for regression task
if task == 0:
    categorical_items = [k for k in range(1,max_items+1)] 
    df_pre = df[selected_categorical_variables]
    df_linear = df[selected_categorical_variables]
    max_total_response = max_items*max(categorical_response_list)

elif task ==1:
    max_index = len(df[df[diagnosis_column] == 1])

    #Combine all items
    all_items = categorical_items + categorical_items_binary + float_items

    # Determine the maximum value for each float item
    max_values = {item: max(df[item]) for item in float_items}

    # Generate the range parameters dynamically based on the max values and user-defined step sizes
    range_params = {item: (0, max_values[item], user_defined_step_sizes[item]) for item in float_items}

    # Generate the cutoffs dictionary dynamically
    cutoffs = {item: np.arange(*range_params[item]) for item in float_items}
    total_variable_length = len(categorical_items+float_items)
    df_neg = df.iloc[df[df[diagnosis_column] == 0].index[:len(df[df[diagnosis_column] == 1])]]
    df_pos = df.iloc[df[df[diagnosis_column] == 1].index[:len(df[df[diagnosis_column] == 1])]]
    df_linear = pd.concat([df_pos,df_neg], axis=0)
    df_pre = df_linear[categorical_items+float_items+categorical_items_binary]    

In [27]:
# Creates new binary indicator columns in the DataFrame df_pre, 
# based on whether values in specific columns of the DataFrame df meet certain threshold conditions. 
# The thresholds are defined by the range of integers in the inner loop (j).

if task == 0:
    for k in selected_categorical_variables:
        for j in range(min(categorical_response_list), len(categorical_response_list)):
            df_pre["Thr{}(I{})".format(j,k)] = df[k][1:].apply(lambda x: 1 if x>=j else 0)
    df_pre.drop(columns = selected_categorical_variables, inplace = True)
    
elif task ==1:
    for k in categorical_items:
        for j in range(min(categorical_response_list), len(categorical_response_list)+1):
            df_pre["Thr{}({})".format(j,k)] = df_pre[k].apply(lambda x: 1 if x>=j else 0)
            df_pre["iThr{}({})".format(j,k)] = df_pre[k].apply(lambda x: 1 if x<j else 0)

    df_pre.drop(columns = categorical_items, inplace = True)

    
    # Generalized loop to create threshold columns
    for var, cut_values in cutoffs.items():
        for j in cut_values:
            # Formatting the column names with properly placed decimal points
            col_name_thr = f"Thr{var.capitalize()}{j:.0f}({var})"
            col_name_i_thr = f"iThr{var.capitalize()}{j:.0f}({var})"

            # Create threshold columns
            df_pre[col_name_thr] = df_pre[var].apply(lambda x: 1 if x >= j else 0)
            df_pre[col_name_i_thr] = df_pre[var].apply(lambda x: 1 if x < j else 0)

    
    df_pre[binary_variables[0]] = df_pre[categorical_items_binary[0]].apply(lambda x: 1 if x==0 else 0)
    df_pre[binary_variables[1]] = df_pre[categorical_items_binary[0]].apply(lambda x: 1 if x==1 else 0)


    df_pre.drop(columns = float_items, inplace = True)
    df_pre.drop(columns = categorical_items_binary, inplace = True)

/var/folders/zn/b4jwzml16gq9cw9j3gdvbwt80000gn/T/ipykernel_10634/2824330735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre["Thr{}(I{})".format(j,k)] = df[k][1:].apply(lambda x: 1 if x>=j else 0)
/var/folders/zn/b4jwzml16gq9cw9j3gdvbwt80000gn/T/ipykernel_10634/2824330735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre["Thr{}(I{})".format(j,k)] = df[k][1:].apply(lambda x: 1 if x>=j else 0)
/var/folders/zn/b4jwzml16gq9cw9j3gdvbwt80000gn/T/ipykernel_10634/2824330735.py:8: SettingWithCopyW

# Prepare the data for training 
Here the data is split into training and testing sets.

In [28]:
if task ==0:
    X = df_pre[1:] # predictor variable
    y = df[categorical_items][:][1:].sum(axis=1).values # target variable
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # split the predictor and target variables (X and y) into training and testing sets 

    alpha = 13 # number of synthetic data points to be added
    added_data_a = dict([(keyname, [1]*alpha) for keyname in x_train.columns])
    tablea = pd.DataFrame(added_data_a)
    x_train = pd.concat([x_train, tablea])

    for i in range(alpha):
        y_train = np.append(y_train, [max_total_response])
    
        
elif task==1:
    X = df_pre/total_variable_length 
    y = df_linear[diagnosis_column]
    x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

# Training 

Initialize and train a symbolic regression model using only the addition function, which means the model will attempt to construct mathematical expressions using only addition operations to fit the training data.

In [29]:
# Define the objective function for regression task
def sym_regression(params):
    p_subtree_mutation, p_hoist_mutation, p_point_mutation = params
    p_crossover=0.7
    # Penalty
    # Ensure the sum of probabilities is <= 1.0
    if p_crossover + p_subtree_mutation + p_hoist_mutation + p_point_mutation > 1.0:
        return 1e6  
    
    est_gp = SR(
        population_size=10000,
        function_set=function_set,
        generations=20,
        stopping_criteria=0.01,
        p_crossover=0.7,
        p_subtree_mutation=p_subtree_mutation,
        p_hoist_mutation=p_hoist_mutation,
        p_point_mutation=p_point_mutation,
        max_samples=0.9,
        verbose=0,
        metric="mean absolute error",
        parsimony_coefficient=0.01,
        random_state=10,
        feature_names=x_train.columns
    )
    
    est_gp.fit(x_train, y_train)
    return -est_gp.score(x_test, y_test)

# Define the objective function for classification task
def sym_classification(params):
    p_subtree_mutation, p_hoist_mutation, p_point_mutation = params
    p_crossover=0.7
    # Penalty
    # Ensure the sum of probabilities is <= 1.0
    if p_crossover + p_subtree_mutation + p_hoist_mutation + p_point_mutation > 1.0:
        return 1e6
    
    est_gp = SC(
        population_size=20000,
        function_set=function_set,
        generations=50,
        stopping_criteria=10**(-4),
        p_crossover=0.7,
        p_subtree_mutation=p_subtree_mutation,
        p_hoist_mutation=p_hoist_mutation,
        p_point_mutation=p_point_mutation,
        max_samples=0.9,
        verbose=0,
        random_state=1,
        feature_names=x_train.columns
    )
    
    est_gp.fit(x_train, y_train)
    predictions_proba = est_gp.predict_proba(x_test)[:, 1]  # Get probabilities for the positive class
    return -roc_auc_score(y_test, predictions_proba)

# Define the search space for Task 0
pbounds_regression = [
    Real(0.0, 0.1, name='p_hoist_mutation'),
    Real(0.0, 0.1, name='p_subtree_mutation'),
    Real(0.0, 0.1, name='p_point_mutation'),
]

# Define the search space for Task 1
pbounds_classification = [
    Real(0.0, 0.1, name='p_hoist_mutation'),
    Real(0.0, 0.1, name='p_subtree_mutation'),
    Real(0.0, 0.1, name='p_point_mutation'),
]


In [30]:
function_set = ['add']

# Run optimization based on the task
if task == 0:
    result = gp_minimize(sym_regression, pbounds_regression, n_calls=20, random_state=1, verbose=True)
    
    # Print and use the best parameters for Task 0
    print("Best parameters for Task 0:", result.x)
    best_params = result.x
    est_gp = SR(
        population_size=10000,
        function_set=function_set,
        generations=20,
        stopping_criteria=0.01,
        p_crossover=0.7,
        p_subtree_mutation=best_params[0],
        p_hoist_mutation=best_params[1],
        p_point_mutation=best_params[2],
        max_samples=0.9,
        verbose=1,
        metric="mean absolute error",
        parsimony_coefficient=0.01,
        random_state=10,
        feature_names=x_train.columns
    )

elif task == 1:
    result = gp_minimize(sym_classification, pbounds_classification, n_calls=20, random_state=1, verbose=True)
    
    # Print and use the best parameters for Task 1
    print("Best parameters for Task 1:", result.x)
    best_params = result.x
    est_gp = SC(
        population_size=20000,
        function_set=function_set,
        generations=100,
        stopping_criteria=10**(-4),
        p_crossover=0.7,
        p_subtree_mutation=best_params[0],
        p_hoist_mutation=best_params[1],
        p_point_mutation=best_params[2],
        max_samples=0.9,
        verbose=1,
        random_state=1,
        feature_names=x_train.columns
    )            

# Fit the model with the optimized parameters
est_gp.fit(x_train, y_train)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 140.6171
Function value obtained: -0.8190
Current minimum: -0.8190
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 143.0240
Function value obtained: -0.8174
Current minimum: -0.8190
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 142.0354
Function value obtained: -0.8044
Current minimum: -0.8190
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 140.2379
Function value obtained: -0.8203
Current minimum: -0.8203
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 138.7180
Function value obtained: -0.8133
Current minimum: -0.8203
Iteration No: 6

SymbolicRegressor(feature_names=Index(['Thr1(I23)', 'Thr2(I23)', 'Thr3(I23)', 'Thr1(I28)', 'Thr2(I28)',
       'Thr3(I28)', 'Thr1(I39)', 'Thr2(I39)', 'Thr3(I39)', 'Thr1(I51)',
       'Thr2(I51)', 'Thr3(I51)', 'Thr1(I58)', 'Thr2(I58)', 'Thr3(I58)',
       'Thr1(I60)', 'Thr2(I60)', 'Thr3(I60)'],
      dtype='object'),
                  function_set=['add'], max_samples=0.9, p_crossover=0.7,
                  p_hoist_mutation=0.03132735169322752,
                  p_point_mutation=0.052454815957287154,
                  p_subtree_mutation=0.08463109166860174,
                  parsimony_coefficient=0.01, population_size=10000,
                  random_state=10, stopping_criteria=0.01, verbose=1)

# Check for overfitting: comparing performance between training and test set

In [13]:
# Predict on training and test sets
if task == 0:
    train_predictions = est_gp.predict(x_train)
    test_predictions = est_gp.predict(x_test)

    train_mae = mean_absolute_error(y_train, train_predictions)
    test_mae = mean_absolute_error(y_test, test_predictions)

    print(f"Training MAE: {train_mae}")
    print(f"Test MAE: {test_mae}")
    
elif task == 1:
    train_predictions = est_gp.predict_proba(x_train)
    test_predictions = est_gp.predict_proba(x_test)
    fpr_sym_train, tpr_sym_train, thresholds_train = roc_curve(y_train, train_predictions[:, 1])
    roc_auc_sym_train = auc(fpr_sym_train, tpr_sym_train)
    fpr_sym_test, tpr_sym_test, thresholds_test = roc_curve(y_test, test_predictions[:, 1])
    roc_auc_sym_test = auc(fpr_sym_test, tpr_sym_test)
    
    print(f"Training AUC: {roc_auc_sym_train}")
    print(f"Test AUC: {roc_auc_sym_test}")


Training MAE: 10.288735017546117
Test MAE: 10.635138766312364


In [14]:
# Threshold ratio for overfitting
if task==0:
    overfit_ratio = 1.5 

    # Check for overfitting
    if test_mae > overfit_ratio * train_mae:
        print("Warning: The model might be overfitting!")
    else:
        print("The model is generalizing well.")

elif task ==1:
    # Set a threshold for AUC difference
    auc_diff_threshold = 0.1

    # Calculate the absolute difference between train and test AUC
    auc_diff = abs(roc_auc_sym_train - roc_auc_sym_test)

    # Issue a warning if the difference exceeds the threshold
    if auc_diff > auc_diff_threshold:
        print(f"Warning: Significant difference in AUC between training and test sets (Difference: {auc_diff:.3f}). The model might be overfitting!")
    else:
        print("The model is generalizing well.")

The model is generalizing well.


# Check for performance on different subsets: Cross-validation

In [ ]:
if task==0:
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform cross-validation with the defined cross-validator
    scores = cross_val_score(est_gp, X, y, cv=cv, scoring='neg_mean_absolute_error')
    mean_cv_score = -np.mean(scores)
    std_cv_score = np.std(scores)
    
    # Print cross-validated AUC result
    print(f"Cross-validated MAE: {mean_cv_score}")
    print(f"Standard deviation of Cross-validated MAE: {std_cv_score}")

elif task ==1:
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Get cross-validated predictions (probabilities) on the entire dataset
    cv_predictions = cross_val_predict(est_gp, X, y, cv=cv, method='predict_proba')

    # Calculate ROC AUC for the cross-validated predictions
    fpr_cv, tpr_cv, _ = roc_curve(y, cv_predictions[:, 1])
    roc_auc_cv = auc(fpr_cv, tpr_cv)

    # Print cross-validated AUC result
    print(f"Cross-validated AUC: {roc_auc_cv}")

In [ ]:
if task ==0:
    print(f"Cross-validated scores: {-scores}")
    print(f"Mean cross-validated score: {mean_cv_score}")
    print(f"Standard deviation of CV scores: {std_cv_score}")

    train_predictions = est_gp.predict(x_train)
    train_mae = mean_absolute_error(y_train, train_predictions)

    print(f"Training MAE: {train_mae}")

    overfit_threshold = 1.5  # You can adjust this value

    # Check for overfitting
    if mean_cv_score > overfit_threshold * train_mae:
        print("Warning: The model might be overfitting! The cross-validation error is significantly higher than the training error.")
    elif std_cv_score > 0.15 * mean_cv_score:  # Check for high variance in CV scores 
        print("Warning: The model might be unstable! High variance in cross-validation performance.")
    else:
        print("The model is generalizing well.")
    
elif task ==1:
    # Calculate AUROC for each fold
    cv_aucs = cross_val_score(est_gp, X, y, cv=cv, scoring='roc_auc')

    # Print the AUROC for each fold
    print(f"AUROC for each fold: {cv_aucs}")

    # Calculate the mean and standard deviation of AUROC across folds
    mean_auc = np.mean(cv_aucs)
    std_auc = np.std(cv_aucs)

    # Print average and standard deviation
    print(f"Mean AUROC: {mean_auc}")
    print(f"Standard deviation of AUROC: {std_auc}")

    # Set thresholds for acceptable performance
    min_auc_threshold = 0.7  # Minimum acceptable AUROC
    max_std_threshold = 0.1  # Maximum acceptable standard deviation of AUROC across folds

    # Check for poor performance
    if mean_auc < min_auc_threshold:
        print(f"Warning: Low mean AUROC ({mean_auc:.3f}) across folds, indicating poor model performance.")

    if std_auc > max_std_threshold:
        print(f"Warning: High variability in AUROC across folds (Standard Deviation: {std_auc:.3f}), indicating model instability.")
    else:
        print("The model is generalizing well.")

# Define a Python dictionary "converter"

The converter dictionary provides a mapping between string keys (e.g., 'sub', 'div', 'mul', 'add') and lambda functions representing arithmetic operations. This dictionary can be used to select the appropriate operation based on a given key and apply it to numerical values.

In [15]:
converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
}

# Evaluate the performance of your symbolic regression model (est_gp)

est_gp.score(x_test, y_test): calculates the R^2 score of the symbolic regression model (est_gp) on the test data (x_test, y_test). The R^2 score measures the proportion of the variance in the target variable that is predictable from the features. Higher R^2 scores indicate a better fit of the model to the data.

next_e: converts the best individual (mathematical expression) learned by the symbolic regression model (est_gp._program) into a SymPy symbolic expression. 

    sympify is a function from SymPy that converts a string representing a mathematical expression into a SymPy
    expression. The locals parameter is used to provide the converter dictionary, which maps mathematical
    operations to corresponding lambda functions.

In [16]:
print('R2 score:',est_gp.score(x_test,y_test))
next_e = sympify(str(est_gp._program), locals = converter)
next_e

R2 score: 0.7375566846336077


16*Thr1(I23) + 13*Thr1(I28) + 15*Thr1(I39) + 8*Thr1(I51) + 13*Thr1(I58) + 17*Thr1(I60) + 6*Thr2(I23) + 5*Thr2(I28) + 7*Thr2(I39) + 5*Thr2(I51) + 13*Thr2(I58) + 8*Thr2(I60) + 8*Thr3(I23) + 10*Thr3(I28) + 5*Thr3(I39) + 7*Thr3(I51) + 8*Thr3(I58) + 5*Thr3(I60) + 10*Thr4(I23) + 12*Thr4(I28) + 7*Thr4(I39) + 8*Thr4(I51) + 13*Thr4(I58) + 10*Thr4(I60) + 3.206

In [17]:
# Process the symbolic expression (next_e) to extract and analyze its terms, coefficients, and values 
# associated with specific variables. 
# After processing, dataframes are created to store the results and then save them to Excel files.

if task ==0:
    e_str = str(next_e).replace(" ", "")
    terms = e_str.split("+")
    variables = selected_categorical_variables
    term_grouped = {str(var): [] for var in variables}
    term_coefs = {str(var): [0]*len(categorical_response_list) for var in variables}
    term_val = {f"Q{var}": [0]*len(categorical_response_list) for var in variables}
        
        
    for term in terms:
        if "I" not in term:
            continue
        left = term.index('I')
        for t in range(len(variables)):
            if term[left+1:-1] == str(variables[t]):
                term_grouped[str(variables[t])].append(term)

    

    for key in term_grouped.keys():
        lst = term_grouped[key]
        for item in lst:
            idx = int(item[item.index("r")+1:item.index("(")]) - 1
            coef = 1
            if item[0]!="T":
                coef = int(item[:item.index("*")])
            term_coefs[key][idx] = coef

        for d in range(len(categorical_response_list)):
            term_val['Q'+key][d] = term_val['Q'+key][d-1] + term_coefs[key][d]

    
elif task==1:
    next_e_str = str(next_e).replace(" ", "")
    terms = re.split('\+|-', next_e_str)

    term_grouped = {}
    term_coefs = {}
    

    for term in terms:
        if '(' in term and ')' in term:
            start_idx = term.index('(') + 1
            end_idx = term.index(')')
            var = term[start_idx:end_idx]
        else:
            if '*' in term:
                var = term.split('*')[-1]
            else:
                continue
        
        if var not in term_grouped:
            term_grouped[var] = []
        term_grouped[var].append(term)
       
        if '*' in term:
            coef = int(term.split('*')[0])
        else:
            coef = 1

        if var not in term_coefs:
            term_coefs[var] = []
        term_coefs[var].append(coef)

    max_numb = {var: 0 for var in term_grouped.keys()}

    for var, terms_list in term_grouped.items():
        for term in terms_list:
            num_match = re.search(r'\d+(?=\()', term)
            if num_match:
                num = int(num_match.group(0))
                if num > max_numb[var]:
                    max_numb[var] = num

    maximum_row = int(max(max_numb.values())) + 1
    term_val = {key: [0]*maximum_row for key in term_coefs.keys()}

    for var, terms_list in term_grouped.items():
        for term in terms_list:
            if '(' not in term:
                if '*' in term:
                    coef = int(term.split('*')[0])
                    for i in range(maximum_row):
                        term_val[var][i] = coef if var in term else 0
                else:
                    for i in range(maximum_row):
                        term_val[var][i] = 1 if var in term else 0
            else:
                num_match = re.search(r'\d+(?=\()', term)
                if num_match:
                    num = int(num_match.group(0))
                else:
                    num = 1
            
                coef = 1
                if '*' in term:
                    coef = int(term.split('*')[0])
                
                if 'iThr' in term:
                    for i in range(0, min(num, maximum_row)):
                        term_val[var][i] += coef

                elif 'Thr' in term:
                    for i in range(num, maximum_row):
                        term_val[var][i] += coef
    
    table0 = pd.DataFrame(term_val)
    print(table0)


In [18]:
if task == 0:
    table0 = pd.DataFrame(term_val)
    table0.to_excel('SCORES/table_temp.xlsx')

    wei = pd.DataFrame(term_coefs)
    wei.to_excel('SCORES/partial_weights.xlsx')

    print(table0)
    print(wei)
    print(table0.iloc[-1].sum())
    
elif task ==1:
    results = {}
    max_values = {}

    
    for column in table0.columns:
        
        column_results = {}
        unique_values = table0[column].unique()

        if pd.api.types.is_numeric_dtype(table0[column]):
            max_values[column] = table0[column].max()

        for unique_value in unique_values:
            indices = []

            for i in range(len(table0)):
                if table0.iloc[i][column] == unique_value:
                    indices.append(i)

            if indices: 
                max_index = max(indices)
                min_index = min(indices)
                column_results[unique_value] = (min_index, max_index)
            else:
                column_results[unique_value] = (None, None)

        results[column] = column_results

    all_scores = []

    for column, column_result in results.items():
        print(f"Variable: {column}")
        actual_max_value = max_values.get(column, None)
        scores = []

        if column in binary_variables:
            unique_value = list(column_result.keys())[0]
            print(f"Score: {unique_value}")
            scores.append([column, "", unique_value])
        else:
            last_range_start = None
            last_range_value = None

            for value, (min_index, max_index) in column_result.items():
                if min_index is not None and max_index is not None:
                    if value == actual_max_value:
                        last_range_start = min_index
                        last_range_value = value
                        break

            for value, (min_index, max_index) in column_result.items():
                if min_index is not None and max_index is not None:
                    if value == last_range_value:
                        range_str = f"{min_index}-{max_index}"
                    elif max_index == maximum_row-1:
                        range_str = f"{min_index}-{max(df[column])}"  
                    else:
                        range_str = f"{min_index}-{max_index}"
                        
                    print(f"Range: {range_str}   Score: {value}")

                    scores.append([column, range_str, value])

        print()  

        all_scores.extend(scores)

    final_score_table = pd.DataFrame(all_scores, columns=['Variable', 'Range', 'Score'])

    final_score_table['Range'] = final_score_table.apply(lambda row: re.sub(r'\d+-$','4-', row['Range']) if row['Variable'].startswith('ISI') else row['Range'], axis=1)

    final_score_table.to_excel('SCORES/score_table_classification.xlsx', sheet_name='Results', index=False)






   Q23  Q28  Q39  Q51  Q58  Q60
0   16   13   15    8   13   17
1   22   18   22   13   26   25
2   30   28   27   20   34   30
3   40   40   34   28   47   40
   23  28  39  51  58  60
0  16  13  15   8  13  17
1   6   5   7   5  13   8
2   8  10   5   7   8   5
3  10  12   7   8  13  10
229


In [19]:
# Mean absolute error
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

In [20]:
if task ==0:
    # Prepare data
    X2 = df_linear[:][1:]
    y2 = df[categorical_items][:][1:].sum(axis=1).values
    x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=10)

    # Deep copy the test data
    xtest = copy.deepcopy(x_test2)

    # Load weights and cumulative term values
    wei = pd.read_excel('SCORES/partial_weights.xlsx')
    table0 = pd.read_excel('SCORES/table_temp.xlsx')

    # Calculate difference between target value and sum of last row in table0
    candy = max_items*len(categorical_response_list) - table0[table0.columns[1:]].iloc[-1].sum()

    # Fine-tune model if difference is not zero
    if candy != 0:
        # Generate cases to explore different adjustments to the coefficients
        cases = list(itertools.product(table0.columns[1:], [0, 1, 2, 3]))
        table_opt = 0
        MAE_opt = 9999999

        # Iterate through cases
        for case in tqdm(cases, desc='진행률'):
            # Deep copy weights
            wgt = copy.deepcopy(wei)

            col, row = case[0], case[1]
            wgt[col[1:]][row] += candy

            # Skip if weight becomes negative
            if wgt[col[1:]][row] < 0:
                continue

            # Calculate cumulative term values with adjusted weights
    
            term_val = {f"Q{var}": [0]*len(categorical_response_list) for var in selected_categorical_variables}
            for key in wei.columns[1:]:
                for d in range(4):
                    term_val['Q' + key][d] = term_val['Q' + key][d - 1] + wgt[key][d]

            weight = pd.DataFrame(term_val)

            # Apply adjusted weights to test data
            dictlist = []
            for key in selected_categorical_variables:
                dic = (key, {k: weight["Q" + str(key)].iloc[k] for k in range(len(categorical_response_list))})
                dictlist.append(dic)
            ttsdic = dict(dictlist)

            for key in selected_categorical_variables:
                xtest[key] = x_test2[key].apply(lambda x: ttsdic[key][x - categorical_response_list[0]])

            # Calculate mean absolute error
            y_pred2 = xtest.sum(axis=1)
            MAE = mae(y_test2, y_pred2)

            # Update optimal solution if MAE is lower
            if MAE < MAE_opt:
                MAE_opt = MAE
                table_opt = copy.deepcopy(weight)
            time.sleep(0.0001)

        # Print and save results
        print(MAE_opt)
        table_opt.to_excel('SCORES/Scores_regression_FineTuned.xlsx')
        print(table_opt)
        
elif task ==1:
    thres_unprocessed = float(terms[-1])
    thres = thres_unprocessed*len(all_items)
    print(f"If total score is greater than {thres:.2f}, then the individual has the disease/disorder.")



진행률:   0%|          | 0/24 [00:00<?, ?it/s]

10.010752688172044
   Q23  Q28  Q39  Q51  Q58  Q60
0   16   13   15    8   13   17
1   22   18   22   13   26   25
2   30   28   27   20   34   30
3   40   40   34   28   47   51


# Saving the Model

In [ ]:
with open('sym_reg_model_mcqi.pkl', 'wb') as file:
    pickle.dump(est_gp, file)